## Libraries and Util

In [1]:
import itertools
import json
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score


def transformToList(inputString):
    s = inputString.translate(str.maketrans({'{': '[', '}': ']', '(': '[', ')': ']'}))
    s = s.replace('[', ' [ ')
    s = s.replace(']', ' ] ')
    s = s.replace(',', ' , ')
    words = s.split()
    output = ""
    for word in words:
        if word=="[" or word=="]" or word==",": output+=word
        else: output+='"'+word+'"'
    out = json.loads(output)
    try:
        a = np.array(out).astype(np.float).tolist()
    except:
        a = out
    return a

class Activation:
    def stepFunction(u):
        if u>0: return 1
        elif u<0: return 0
        else: return 0.5


class Utility:
    def augmentArray(a, value=1, position=0):
        return np.insert(a, position, value, axis=len(a.shape)-1)
    
    def sampleNorm(data, target, mainClass = 1):
        if len(data) != len(target):
            print("incompatible array sizes - sampleNorm")
            return
        for i in range(len(target)):
            if target[i] != mainClass:
                data[i] = [-x for x in data[i]]
        return data 

## Week 1

### Confusion Matrix and Metrics

#### Code

In [ ]:
class Week1:
    def basic_metrics(y_true, y_pred, class_names, normalize=False):
        cm = confusion_matrix(y_true, y_pred)
        cm = cm[:,::-1][::-1]
        np.set_printoptions(precision=4)

        title='Confusion matrix'
        cmap=plt.cm.Blues
        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        plt.title(title)
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        if normalize:
            cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

        thresh = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, cm[i, j],
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

        print(classification_report(y_true, y_pred, target_names=class_names[::-1],digits=4))
        plt.show()
        
    def setParams():
        class_names = transformToList(input("Enter names of classes in descending order eg [Two, One, Zero]:  "))
        y_true = transformToList(input("Enter true value of labels eg [1,1,0,1,0,1,1,2] 1D:  "))
        y_pred = transformToList(input("Enter predicted value of labels eg [1,0,1,1,0,1,0,2] 1D:  "))
        return class_names, y_true, y_pred

#### Run

In [ ]:
class_names, y_true, y_pred = Week1.setParams()
Week1.basic_metrics(y_true, y_pred, class_names)

## Week 2

### Batch Perceptron Learning

#### Code

In [ ]:
class BatchPerceptronLearning:
    def fit(X, Y, a, n):
        # ------------------------------------------------------------------------------------
        # Applying Sample Normalisation:
        Norm_Y = []

        for x, y in zip(X, Y):

            # If the sample belongs to the class with label 2 or -1 (Check dataset in question to see how formatted):
            if y != 1:
                x = [i * -1 for i in x]
                x.insert(0, -1)
                Norm_Y.append(x)
            else:
                x.insert(0, 1)
                Norm_Y.append(x)

        print("Vectors used in Batch Perceptron Learning Algorithm:\n {}\n".format(Norm_Y))

        # ------------------------------------------------------------------------------------
        # Batch Perceptron Learning Algorithm:

        epoch = 1

        while True:

            updating_samples = []
            print("Epoch {}".format(epoch))

            for count, i in enumerate(range(len(Norm_Y))):

                # Knowing which value of a to use. If it is the first iteration, than use the given parameters in the 
                # question:
                a_prev = a
                print("The value of a used is {}".format(a_prev))
                y_input = Norm_Y[i]
                print("y Value used for this iteration is: {}".format(y_input))

                # Equation -> g(x) = a^{t}y
                ay = np.dot(a, y_input)
                print("The value of a^t*y for this iteration is: {}".format(ay))


                # Checking if the sample is misclassified or not:

                # If sample is misclassified:
                if ay <= 0:

                    # If this is the first sample in the epoch, add the previous value of a to the list of samples used 
                    # for the update to perform summation at the end of the epoch:
                    if count == 0:
                        print("This sample is misclassified. This sample will be used in update.\n")
                        updating_samples.append(np.array(a))
                        updating_samples.append(np.array(y_input))

                    # If sample is misclassified and IS NOT the first sample in the epoch:
                    else:
                        print("This sample is misclassified. This sample will be used in update.\n")
                        updating_samples.append(np.array(y_input))

                # If sample is classified correctly:
                else: 

                    # If first sample in the epoch, append the previous value of a to the updating samples list:
                    if count == 0:
                        updating_samples.append(np.array(a))
                        print("This sample is classified correctly.\n")
                    else:
                        print("This sample is classified correctly.\n")

            # Calculating new value of a after having gone through all of the samples in the dataset since it is Batch Learning.
            a_update_val = n * sum(updating_samples)

            # If Block to check whether learning has converged. If we have gone through all the data without needing 
            # to update the parameters, we can conclude that learning has converged.
            if len(updating_samples) <= 1:
                print("\nLearning has converged.")
                print("Required parameters of a are: {}".format(a))
                break

            # Updating a using our new value of a:
            a = a_update_val
            print("\nNew Value of a^t is: {}.\n".format(a))

            epoch += 1
        
    def setParams():
        X_train = transformToList(input("Enter features eg [[1, 5], [2, 5], [4, 1], [5, 1]] 2D:  "))
        y_train = transformToList(input("Enter labels eg [1, 1, 2, 2] 1D:  "))
        a = transformToList(input("Enter a. Usually [1, w1, w2...] eg [-25, 6, 3] 1D:  "))
        lr = float(input("Enter the learning rate eg 1.  "))
        print(type(X_train))
        return X_train, y_train, a, lr

#### Run

In [ ]:
X_train, y_train, a, lr = BatchPerceptronLearning.setParams()
BatchPerceptronLearning.fit(X_train, y_train, a, lr)

### Sequential Perceptron Learing

#### Code

In [ ]:
class SequentialPerceptronLearning:
    def fit(X, Y, a, n):
        # ------------------------------------------------------------------------------------
        # Applying Sample Normalisation:
        Norm_Y = []

        for x, y in zip(X, Y):

            # If the sample belongs to the class with label 2 or -1 (Check dataset in question to see how formatted):
            if y != 1:
                x = [i * -1 for i in x]
                x.insert(0, -1)
                Norm_Y.append(x)
            else:
                x.insert(0, 1)
                Norm_Y.append(x)

        print("Vectors used in Sequential Perceptron Learning Algorithm:\n {}\n".format(Norm_Y))


        # ------------------------------------------------------------------------------------
        # Sequential Perceptron Learning Algorithm:

        epoch = 1

        while True:

            updating_samples = []
            print("Epoch {}".format(epoch))

            # Keeping track of how many samples are correctly classified. If this variable reaches 
            # the value that is equal to the size of the dataset (len), than we know that learning 
            # has converged:
            correctly_classified_counter = 0

            # Going through all of the samples in the dataset one-by-one:
            for i in range(len(Norm_Y)):

                # This chooses which weight to use for an iteration. If first iteration, uses given starting weight 
                # as described in question:
                a_prev = a
                print("The value of a used is {}".format(a_prev))

                # Selecting sample to use:
                y_input = Norm_Y[i]
                print("y Value used for this iteration is: {}".format(y_input))

                # Equation -> g(x) = a^{t}y
                ay = np.dot(a, y_input)
                print("The value of a^t*y for this iteration is: {}".format(ay))


                # Checking if the sample is misclassified or not:

                # If sample is misclassified:
                if ay <= 0:

                    print("This sample is misclassified. This sample will be used in update.\n")
                    updating_samples.append(np.array(a))
                    updating_samples.append(np.array(y_input))

                    # Calculating new value of a using update rule for Sequential Perceptron Learning Algorithm:
                    a_update_val = n * sum(updating_samples)

                    a = a_update_val
                    print("\nNew Value of a^t is: {}.\n".format(a))

                # If the sample is correctly classified:
                else: 
                    print("This sample is classified correctly.\n")
                    correctly_classified_counter += 1
                    pass

                # Reset sample to add for update to occur:
                updating_samples = []

            # If Block to check whether learning has converged. If we have gone through all the data without needing 
            # to update the parameters, we can conclude that learning has converged.
            if correctly_classified_counter == len(Norm_Y):
                print("\nLearning has converged.")
                print("Required parameters of a are: {}.".format(a))
                break

            epoch += 1
        
    def setParams():
        X_train = transformToList(input("Enter features eg [[0, 2], [1, 2], [2, 1], [-3, 1], [-2, -1], [-3, -2]] 2D:  "))
        y_train = transformToList(input("Enter labels eg [1, 1, 1, -1, -1, -1] 1D:  "))
        a = transformToList(input("Enter a. Usually [1, w1, w2...] eg [1, 0, 0] 1D:  "))
        lr = float(input("Enter the learning rate eg 1.  "))
        return X_train, y_train, a, lr

#### Run

In [ ]:
X_train, y_train, a, lr = SequentialPerceptronLearning.setParams()
SequentialPerceptronLearning.fit(X_train, y_train, a, lr)

### Multi-class Perceptron learning

#### Code

In [3]:
class SequentialMultiClassPerceptronLearning:
    def fit(X,y, a, eta):
    # N is the number of exemplars provided in the question
    # augmented_matrix is the augmented feature vector from the question
    # eta is the learning rate given in the question
    # omega is an array containing all the output classes of the feature vectors
        augmented_matrix = np.array(X).T
    # counter which keeps track of cases where winner_class == omega[index]
        N_counter = 0
        
        omega = np.array(y).astype(int).tolist()
        N = augmented_matrix.shape[1]
        number_of_classes = len(set(omega))
        number_of_features = augmented_matrix.shape[0]
        
        print(f'class nb:{number_of_classes} and number of features {number_of_features} and number of shit {N}')
        # Step 2. Initialise aj for each class
        at = np.array(a)
        #at = np.zeros((number_of_classes, number_of_features))
        
        for i in range(0, 15):
            print('Iteration: ', i+1)
            # Step 3. Find values of g1, g2 and g3 and then select the arg max of g
            index = i % N

            # Print updated a^t value
            print('a^t:')
            print(at)

            # Compute g value
            g = np.empty([number_of_classes])
            for i in range(len(g)):
                print('Calculation of g values..........')
                print('a^t is:', at[i])
                print('Index is:', index)
                print('Aug matrix is:', augmented_matrix[:, index])
                g[i] = at[i] @ augmented_matrix[:, index]

            print('g1 | g2 | g3')
            print(g)

            # Step 4. Select the winner
            # Logic for 0,0,0 case and similar ones where 2 gs can produce max value
            seen = []
            bRepeated = False
            # Check if there are multiple max values, and assign the winner class accordingly
            for number in g:
                if number in seen:
                    bRepeated = True
                    print("Number repeated!")
                    m = max(g)
                    temp = [index for index, j in enumerate(g) if j == m]
                    winner_class = max(temp) + 1
                else:
                    seen.append(number)
            # If all g values are unique, simply select the max value's class as the winner
            if(bRepeated == False):
                g = g.tolist()
                arg_max = max(g)
                winner_class = g.index(arg_max) + 1

            print('Winner class = ', winner_class,
                  ', and actual class is:', omega[index])

            # Compare winnner to actual class
            if(winner_class != omega[index]):
                # Step 4. Apply the update rule as per the algorithm

                # Increment the actual class value which is incorrectly classified
                at[omega[index]-1] = at[omega[index]-1] + \
                    eta * augmented_matrix[:, index]
                print('New loser value:', at[omega[index]-1])

                # Penalize the wrongly predicted Winner class
                at[winner_class-1] = at[winner_class-1] - \
                    eta * augmented_matrix[:, index]
                print('New winner value:', at[winner_class-1])

                # Reset counter to 0
                N_counter = 0
            else:
                print('No update is performed!')
                # Increment convergence counter which keeps track of cases where winner_class == omega[index]
                N_counter += 1
                if(N_counter == N):  # check for convergence
                    print('Value of N = ', N)
                    print('Value of N_counter = ', N_counter)
                    print('Learning has converged, so stopping...')
                    print('Final values of a^t after update....')
                    print('at')
                    print(at)
                    break
                print('N counter value = ', N_counter)
            print('at')
            print(at)
            print('=========================================================')


    def setParams():
        X_train = transformToList(input("Enter features eg [[1,0, 2], [1, 1, 2], [1, 2, 1], [1, -3, 1], [1, -2, -1], [1, -3, -2]] 2D:  "))
        y_train = transformToList(input("Enter labels eg [1, 1, 1, -1, -1, -1] 1D:  "))
        a = transformToList(input("Enter a. Usually [1, w1, w2...] eg [1, 0, 0] 1D:  "))
        lr = float(input("Enter the learning rate eg 1.  "))
        print(X_train, y_train, a, lr)
        return X_train, y_train, a, lr


#### Run

In [6]:
X_train, y_train, a, lr = SequentialMultiClassPerceptronLearning.setParams()
SequentialMultiClassPerceptronLearning.fit(X_train, y_train, a, lr)

Enter features eg [[1,0, 2], [1, 1, 2], [1, 2, 1], [1, -3, 1], [1, -2, -1], [1, -3, -2]] 2D:  [[1,1,1],[1,2,0],[1,0,2],[1,-1,1],[1,-1,-1]]


<ipython-input-1-d1c9629e3f07>:21: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  a = np.array(out).astype(np.float).tolist()


Enter labels eg [1, 1, 1, -1, -1, -1] 1D:  [1,1,2,2,3]
Enter a. Usually [1, w1, w2...] eg [1, 0, 0] 1D:  [[0,0,0],[0,0,0],[0,0,0]]
Enter the learning rate eg 1.  1
[[1.0, 1.0, 1.0], [1.0, 2.0, 0.0], [1.0, 0.0, 2.0], [1.0, -1.0, 1.0], [1.0, -1.0, -1.0]] [1.0, 1.0, 2.0, 2.0, 3.0] [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]] 1.0
class nb:3 and number of features 3 and number of shit 5
Iteration:  1
a^t:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Calculation of g values..........
a^t is: [0. 0. 0.]
Index is: 0
Aug matrix is: [1. 1. 1.]
Calculation of g values..........
a^t is: [0. 0. 0.]
Index is: 0
Aug matrix is: [1. 1. 1.]
Calculation of g values..........
a^t is: [0. 0. 0.]
Index is: 0
Aug matrix is: [1. 1. 1.]
g1 | g2 | g3
[0. 0. 0.]
Number repeated!
Number repeated!
Winner class =  3 , and actual class is: 1
New loser value: [1. 1. 1.]
New winner value: [-1. -1. -1.]
at
[[ 1.  1.  1.]
 [ 0.  0.  0.]
 [-1. -1. -1.]]
Iteration:  2
a^t:
[[ 1.  1.  1.]
 [ 0.  0.  0.]
 [-1. -1. -1.]]
Calc

In [ ]:

    # Set input variables for the sequential_multiclass_perceptron_learning function
    N = 5  # N refers to the number of exemplars in the input dataset
    eta = 1
    # Input matrix from the question
    augmented_matrix = np.array(
        [[1, 1, 1, 1, 1], [1, 2, 0, -1, -1], [1, 0, 2, 1, -1]])
    omega = np.array([1, 1, 2, 2, 3])  # Class labels from the question
    number_of_classes = 3
    number_of_features = 3

    # Call function
    SequentialMultiClassPerceptronLearning.fit(
        N, augmented_matrix, eta, omega, number_of_classes, number_of_features)

### Moore-Penrose Pseudoinverse

#### Code

In [ ]:
class Pseudoinverse:
    def fit(X, Y, b):
        # ------------------------------------------------------------------------------------
        # Applying Sample Normalisation:
        Norm_Y = []

        for x, y in zip(X, Y):

            # If the sample belongs to the class with label 2 or -1 (Check dataset in question to see how formatted):
            if y == -1 or y == 2:
                x = [i * -1 for i in x]
                x.insert(0, -1)
                Norm_Y.append(x)
            else:
                x.insert(0, 1)
                Norm_Y.append(x)

        print("Vectors used in Pseudoinverse operation to calculate parameters of linear discriminant function:\n {}\n".format(Norm_Y))

        # ------------------------------------------------------------------------------------
        # Initialising Y Matrix:
        Y_matrix = []

        # Adding each normalised sample in dataset to Y Matrix:
        for i in range(len(Norm_Y)):
            Y_matrix.append(Norm_Y[i])
        Y_matrix = np.array(Y_matrix)
        print("y Matrix being used:\n {}\n".format(Y_matrix))

        # Calculating pseudo-inverse of Y Matrix:
        pseudo_inv_matrix = np.linalg.pinv(Y_matrix)
        print("Pseudo-inverse Matrix is:\n {}\n".format(pseudo_inv_matrix))

        # Multiplying Pseudo-inverse matrix by given margin vector in question:
        a = np.matmul(pseudo_inv_matrix, b)
        print("a is equal to:\n {}\n".format(a))

        correct_classification = 0

        # Checking if classifications are correct:

        for sample in Norm_Y:
            ay = np.dot(sample, a)
            print("\ng(x) for sample {} is {}".format(sample, ay))

            # Sample is correctly classified if ay is positive:    
            if ay > 0:
                print("Sample has been correctly classified.")
                correct_classification += 1

        if correct_classification == len(Norm_Y):
            print("\nAll samples are classified correctly which means that discriminant function parameters are correct.")

        else:
            print("\nSome samples are misclassified.")
        
    def setParams():
        X_train = transformToList(input("Enter features eg [[0, 2], [1, 2], [2, 1], [-3, 1], [-2, -1], [-3, -2]] 2D:  "))
        y_train = transformToList(input("Enter labels eg [1, 1, 1, -1, -1, -1] 1D:  "))
        b = transformToList(input("Enter b eg [1, 1, 1, 1, 1, 1] 1D:  "))
        return X_train, y_train, b

#### Run

In [ ]:
X_train, y_train, b = Pseudoinverse.setParams()
Pseudoinverse.fit(X_train, y_train, b)

### Sequential Widrow Hoff Learning

#### Code

In [ ]:
class SequentialWidrowHoff:
    def fit(X, Y, a, b, n, iterations):
        # ------------------------------------------------------------------------------------
        # Applying Sample Normalisation:
        Norm_Y = []

        for x, y in zip(X, Y):

            # If the sample belongs to the class with label 2 or -1 (Check dataset in question to see how formatted):
            if y == -1 or y == 2:
                x = [i * -1 for i in x]
                x.insert(0, y)
                Norm_Y.append(x)
            else:
                x.insert(0, y)
                Norm_Y.append(x)

        print("Vectors used in Sequential Widrow-Hoff Learning Algorithm:\n {}\n".format(Norm_Y))


        # ------------------------------------------------------------------------------------
        # Sequential Widrow-Hoff Learning Algorithm

        # Epoch for-loop:
        for o in range(int(iterations / len(Norm_Y))):

            # This for-loop goes through each sample one-by-one:
            for i in range(len(Norm_Y)):

                # Value of a to use. If first iteration, then uses parameters given in question:

                a_prev = a

                # Which sample to use:
                y_input = Norm_Y[i]
                print("Sample used for this iteration is: {}".format(y_input))

                # Equation -> g(x) = a^{t}y
                ay = np.dot(a, y_input)
                print("g(x) = {}".format(ay))

                # Calculating the values for update:
                update = np.zeros(len(y_input))
                for j in range(len(y_input)): 

                    # Applying Update Rule of Sequential Widrow-Hoff Learning Algorithm:
                    update[j] = n * (b[i] - ay) * y_input[j]

                # Adding update to a:
                a = np.add(a, update)
                print("New Value of a^t is: {}\n".format(a))

        print("Gone through all of the iterations as asked for in question.")
        
    def setParams():
        X_train = transformToList(input("Enter features eg [[0, 2], [1, 2], [2, 1], [-3, 1], [-2, -1], [-3, -2]] 2D:  "))
        y_train = transformToList(input("Enter labels eg [1, 1, 1, -1, -1, -1] 1D:  "))
        a = transformToList(input("Enter a. Usually [1, w1, w2...] eg [1, 0, 0] 1D:  "))
        b = transformToList(input("Enter b eg [1, 0.5, 1.5, 1.5, 1.5, 1] 1D:  "))
        lr = float(input("Enter the learning rate eg 0.1:  "))
        epochs = int(input("Enter the number of epochs eg 12:  "))
        return X_train, y_train, a, b, lr, epochs

#### Run

In [ ]:
X_train, y_train, a, b, lr, epochs = SequentialWidrowHoff.setParams()
SequentialWidrowHoff.fit(X_train, y_train, a, b, lr, epochs)

## Week 3

### Neuron Output (with heavy side function)

#### Code

In [ ]:
class NeuronOutput:
    def fit(weight, threshold, x):
        summation = []
        for i in range(len(x)):
            summation.append(weight[i] * x[i])

        summation = np.sum(summation, 0) - threshold

        # Find output of neuron by applying heaviside function with given threshold:
        output = np.heaviside(summation, threshold)
        print("Output of neuron with input {} is {}.".format(x, output))
        
    def setParams():
        #This script is based off of Question 2 in Tutorial 3
        x = transformToList(input("Enter single sample/input eg [0.1, -0.5, 0.4] 1D:  "))
        weights = transformToList(input("Enter weigths from one layer to another eg [0.1, -5, 0.4] 1D:  "))
        threshold = float(input("Enter the threshold value eg 0:  "))
        return weights, threshold, x

#### Run

In [ ]:
weights, threshold, x = NeuronOutput.setParams()
NeuronOutput.fit(weights, threshold, x)

### Batch and Sequential Delta Learning

#### Code

In [ ]:
class DeltaLearningModel:
    def __init__(self):
        self.w = None

    def fit(self, features, target, weights, threshold, lr, epochs, type = "S", display=True):
        x = Utility.augmentArray(features.astype(np.float), 1)
        self.w = Utility.augmentArray(weights.astype(np.float), -threshold)
        t = target.astype(np.float)
        print("Score Before Training : ", round(self.score(features, target)*100, 1), "%")
        
        count = 0
        for e in range(0,epochs):
            error=0
            if count>=epochs: break;
            for i in range(0,len(x)):
                y = Activation.stepFunction(x[i].dot(self.w))
                error += (t[i]-y)*x[i]
                
                if type.upper()=="S":
                    self.w = self.w+lr*error
                    if display:
                        count+=1
                        print(count, "\t H(wx) = ", round(y,4), "\t delta w or n(t-y)x = ", np.round(lr*error,4), "\t w = ", np.round(self.w,4))
                    error=0
            if type.upper()=="B":
                count+=1
                self.w = self.w+lr*error
                if display:
                    print(count, "\t Weight change = ", np.round(error,4), "\t w = ", np.round(self.w,4))
                
                
        
        print("Score After Training : ", round(self.score(features, target)*100, 1), "%")
        print("Final Weights (w) = ", self.w)
        
    def predict(self, X_values):
        x = Utility.augmentArray(X_values.astype(np.float), 1)
        myfunc = lambda t: Activation.stepFunction(t.dot(self.w))
        return np.apply_along_axis(myfunc, 1, x)
    
    def score(self, X_test, y_target):
        y_test = self.predict(X_test)
        return np.sum(y_test == y_target)/len(y_target)
    
    def setParams(self):
        batchOrSeq = str(input("'s' for sequential or 'b' for batch.  "))
        X_train = np.array(transformToList(input("Enter features eg [[0, 0], [0, 1], [1, 0], [1, 1]] 2D:  "))).astype(np.float)
        y_train = np.array(transformToList(input("Enter labels eg [0, 0, 0, 1] 1D:  "))).astype(np.float)
        initWeights = np.array(transformToList(input("Enter weights [w1, w2, ..., wd] eg [1, 1] 1D:  "))).astype(np.float)
        threshold = float(input("Enter the threshold value eg -0.5.  "))
        learningRate = float(input("Enter the learning rate eg 1.  "))
        epochs = int(input("Enter the number of epochs eg 4.  "))
        return X_train, y_train, initWeights, threshold, learningRate, epochs, batchOrSeq
    
    def newPred(self):
        ispredict = str(input("\nDo you want to predict using trained model? y/n.  "))
        while ispredict.upper()=="Y":
            x_pred = np.array(transformToList(input("Enter features eg [[0, 0], [0, 1], [1, 0], [1, 1]] 2D:  ")))
            print("Predicted values: ", self.predict(x_pred))
            ispredict = str(input("\nDo you want to predict using trained model? y/n.  "))

#### Run

In [ ]:
model = DeltaLearningModel()
X_train, y_train, initWeights, threshold, learningRate, epochs, batchOrSeq = model.setParams()
model.fit(X_train, y_train, initWeights, threshold, learningRate, epochs, batchOrSeq)
model.newPred()

### Softmax (For Competitive Learning Networks)

#### Code

In [ ]:
class Softmax:
    def fit(x, b=1):
        print("\nSoftmax of array: ", x, "\n")
        print(np.exp(x*b) / np.sum(np.exp(x*b), axis=0))
    
    def setParams():
        a = np.array(transformToList(input("Enter array eg [0.34, 0.73, -0.61] 1D:  "))).astype(np.float)
        beta = float(input("Enter Beta value (1 if not specified).  "))
        return a, beta

#### Run


In [ ]:
a, beta = Softmax.setParams()
Softmax.fit(a, beta)

### Negative Feedback Networks

#### Code

In [ ]:
class NegativeFeddbackNetwork:
    def fit(weights, iterations, x, alpha, activations):
        prev_activations = activations
        iteration = 1

        for i in range(iterations):

            print("Iteration {}".format(iteration))

            # Following block deals with calculating first equation: e = x - W^{T}y
            wT = np.array(weights).T
            wTy = np.dot(wT, activations)
            print("value of wTy {}".format(wTy))

            eT = x - wTy
            print("eT: {}".format(eT))
            e = np.array(eT).reshape((3, 1))

            # The following lines deal with calculating the update: y <- y + \alpha*W*e
            We = np.dot(weights, e)
            We = [j for i in We for j in i]
            print("We: {} ".format(We))

            alphaWe = np.dot(alpha, We)

            # Doing the actual update using the second equation:
            y = activations + alphaWe
            print("Value of y: {}\n".format(y))

            activations = y

            iteration += 1

        print("\nAfter {} iterations, the activation of the output neurons is equal to {}".format(iterations, activations))
        
    def setParams():
        print("This script is based off of Question 7 in Tutorial 3")
        x = transformToList(input("Enter single sample/input eg [1, 1, 0] 1D:  "))
        weights = transformToList(input("Enter weigths from one layer to another eg [[1, 1, 0], [1, 1, 1]] 2D:  "))
        activation = transformToList(input("Enter output layer initial activation eg [0, 0] 1D:  "))
        lr = float(input("Enter the learning rate/alpha eg 0.25.  "))
        epochs = int(input("Enter the number of epochs eg 5.  "))
        return weights, epochs, x, lr, activation

#### Run

In [ ]:
weights, epochs, x, lr, activation = NegativeFeddbackNetwork.setParams()
NegativeFeddbackNetwork.fit(weights, epochs, x, lr, activation)

## Week 4

## Week 5

### Activation Function Operations

#### Code

In [ ]:
class ActivationFunctionOperations:
    '''
    Based on Question 4 in Tutorial 5:
    The following array show the output produced by a mask in a convolutional layer of a CNN.
              [[1, 0.5, 0.2], 
     net_j =  [-1, -0.5, -0.2], 
              [0.1, -0.1, 0]]
    Calculate the values produced by the application of the following activation functions:
    '''

    def fit(net_j, activation_function, a = 0.1, threshold = 0.1, heaviside_0 = 0.5):

        new_array = []

        if activation_function == 'ReLU':

            for row in net_j:
                temp_array = []
                for i in row:

                    # threshold:
                    if i >= 0:
                        temp_array.append(i)
                    else:
                        temp_array.append(0)
                new_array.append(temp_array)

            print(activation_function, ":  ", new_array)

        elif activation_function == 'LReLU':

            for row in net_j:
                temp_array = []
                for i in row:

                    # threshold:
                    if i >= 0:
                        temp_array.append(i)
                    else:
                        temp_array.append(round(a * i, 2))
                new_array.append(temp_array)

            print(activation_function, ":  ", new_array)

        elif activation_function == 'tanh':

            for row in net_j:
                temp_array = []
                for i in row:

                    # Using equation of tanh activation function:
                    temp_array.append(round((math.e**i - math.e ** -i) / (math.e**i + math.e ** -i), 5)) 
                new_array.append(temp_array)

            print(activation_function, ":  ", new_array)

        elif activation_function == 'heaviside':

            for row in net_j:
                temp_array = []
                for i in row:

                    # subtracts threshold away from each value:
                    i = i - threshold

                    # applies heaviside function to value:
                    temp_array.append(np.heaviside(i, heaviside_0))

                new_array.append(temp_array)

            print(activation_function, ":  ", new_array)
            
    def setParams():
        
        activation_function = "ReLU"
        a = 0.1
        threshold = 0.1
        heaviside_0 = 0.5
        
        net_j = transformToList(input("Enter matrix to perform activation on eg [[1, 0.5, 0.2], [-1, -0.5, -0.2], [0.1, -0.1, 0]] 2D:  "))
        typeActivation = str(input("'ReLU', 'LReLU', 'tanh' or 'heaviside' "))
        if typeActivation.upper()=="RELU":
            activation_function = "ReLU"
        elif typeActivation.upper()=="LRELU":
            activation_function = "LReLU"
            a = float(input("Enter the a value eg 0.1.  "))
        elif typeActivation.lower()=="tanh": 
            activation_function = "tanh"
        elif typeActivation.lower()=="heaviside": 
            activation_function = "heaviside"
            threshold = float(input("Enter the threshold value eg 0.1.  "))
            heaviside_0 = float(input("Enter the H(0) value eg 0.5.  "))
        else:
            print("wrong input")
            
        return net_j, activation_function, a, threshold, heaviside_0

#### Run

In [ ]:
net_j, activation_function, a, threshold, heaviside_0 = ActivationFunctionOperations.setParams()
ActivationFunctionOperations.fit(net_j, activation_function, a, threshold, heaviside_0)

## Week 6

## Week 7

## Week 8

### SVM(to find lambda, weights and margin)

#### Code

In [ ]:
class SVM:
    def fit(X, y, support_vectors, support_vector_class):
        X = np.array(X)
        y = np.array(y)

        print("-"*100)
        w = []
        for idx in range(len(support_vectors)):
            w.append(support_vectors[idx] * support_vector_class[idx])
        w = np.array(w)
        eq_arr = []
        for idx, sv in enumerate(support_vectors):
            tmp = ((w @ sv) * support_vector_class[idx])
            tmp = np.append(tmp, [support_vector_class[idx]])
            eq_arr.append(tmp)
        eq_arr.append(np.append(support_vector_class, [0]))
        rhs_arr = [1] * len(support_vector_class)
        rhs_arr.extend([0])
        rhs_arr = np.array(rhs_arr)
        ans = rhs_arr @ np.linalg.inv(eq_arr)
        print("lambda and w_0 values are ", ans)
        final_weight = []
        for idx in range(w.shape[0]):
            final_weight.append(w[idx] * ans[idx])
        final_weight = np.array(final_weight)
        final_weight = np.sum(final_weight, axis=0)
        print("Weights: ")
        print(final_weight)
        print("Margin: ")
        print(2/np.linalg.norm(final_weight))
        print("-"*100)
            
    def setParams():
        X = transformToList(input("Enter features eg [[3, 1], [3, -1], [7, 1], [8, 0], [1, 0], [0, 1], [-1, 0], [-2, 0]] 2D:  "))
        y = transformToList(input("Enter labels eg [1, 1, 1, 1, -1, -1, -1, -1] 1D:  "))
        support_vectors = np.array(transformToList(input("Enter features eg [[3, 1], [3, -1], [1, 0]] 2D:  "))).astype(np.float)
        support_vector_class = np.array(transformToList(input("Enter labels eg [1, 1, -1] 1D:  "))).astype(np.float)
            
        return X, y, support_vectors, support_vector_class

#### Run

In [ ]:
X, y, support_vectors, support_vector_class = SVM.setParams()
SVM.fit(X, y, support_vectors, support_vector_class)

## Week 9

## Week 10